In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('database.sqlite')

In [3]:
query = "SELECT * FROM sqlite_master"

df_schema = pd.read_sql_query(query, conn)

df_schema.tbl_name.unique()

array(['sqlite_sequence', 'Player_Attributes', 'Player', 'Match',
       'League', 'Country', 'Team', 'Team_Attributes'], dtype=object)

1. Which team scored the most points when playing at home?

In [16]:
query1 = """
    SELECT Team.team_long_name, DT_goals.goals
    FROM
    (SELECT home_team_api_id, SUM(home_team_goal) as goals 
    FROM Match 
    GROUP BY home_team_api_id
    ORDER BY goals DESC
    LIMIT 1) AS DT_goals
    LEFT JOIN Team ON Team.team_api_id = DT_goals.home_team_api_id
    ;
"""

df_teams = pd.read_sql_query(query1, conn)
df_teams

,team_long_name,goals
0,Real Madrid CF,505


2. Did this team also score the most points when playing away?

In [18]:
query2 = """
    SELECT Team.team_long_name, DT_goals.goals
    FROM
    (SELECT away_team_api_id, SUM(away_team_goal) as goals 
    FROM Match 
    GROUP BY away_team_api_id
    ORDER BY goals DESC
    LIMIT 1) AS DT_goals
    LEFT JOIN Team ON Team.team_api_id = DT_goals.away_team_api_id
    ;
"""

df_2 = pd.read_sql_query(query2, conn)
df_2

,team_long_name,goals
0,FC Barcelona,354


Answer: Another team got the most points when playing away

3. How many matches resulted in a tie?

In [21]:
query3 = """
    SELECT COUNT(*) 
    FROM Match 
    WHERE home_team_goal = away_team_goal;
"""

df_3 = pd.read_sql_query(query3, conn)
df_3

,COUNT(*)
0,6596


4. How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [30]:
query4_last = """
    SELECT COUNT(*) 
    FROM Player 
    WHERE player_name LIKE '%Smith';
"""

df_4_last = pd.read_sql_query(query4_last, conn)
df_4_last

,COUNT(*)
0,18


In [33]:
query4_any = """
    SELECT COUNT(*)
    FROM Player 
    WHERE player_name LIKE '%Smith%';
"""

df_4_any = pd.read_sql_query(query4_any, conn)
df_4_any

,COUNT(*)
0,18


5. What was the median tie score? Use the value determined in the previous question for the number of tie games. Hint: PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the WITH command to store stepwise results as a table and then operate on these results.

In [143]:
query5 = """

WITH DT1 AS(
    SELECT *, ROW_NUMBER() OVER(ORDER BY home_team_goal) num
    FROM Match 
    WHERE home_team_goal = away_team_goal 
    ORDER BY Match.home_team_goal
    ),   
DT2 AS(
    SELECT COUNT(*)/2 as median
    FROM Match 
    WHERE home_team_goal = away_team_goal
    )   

SELECT DT1.*  
FROM DT1, DT2
WHERE DT1.num = DT2.median
LIMIT 1
;
"""

df_5 = pd.read_sql_query(query5, conn)
df_5

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,num
0,11165,10257,10257,2010/2011,22,2011-01-30 00:00:00,888378,9888,9880,1,...,2.15,3.2,4,2.1,3.2,3.5,2.15,3.2,3.3,3298


6. What percentage of players prefer their left or right foot? Hint: Calculate either the right or left foot, whichever is easier based on how you setup the problem.

In [104]:
query6 = """

WITH DT1 AS(
    SELECT COUNT(*) as count_right
    FROM Player_Attributes
    WHERE preferred_foot = 'right'
    ),   
DT2 AS(
    SELECT COUNT(*) as count_all
    FROM Player_Attributes
    )   

SELECT CAST (DT1.count_right AS float) / CAST(DT2.count_all AS float) as pr_right
FROM DT1, DT2
;
"""

df_6 = pd.read_sql_query(query6, conn)
df_6

,pr_right
0,0.752313
